### 1변량 데이터의 t검정
* 대상 : 평균값
* 판단하는 것 : 평균값이 어떤 값과 다른지 얘기할 수 있는지 여부

평균과 유의미한 차이가 있다고 판단하려면 다음과 같은 조건이 전제되어야 한다.  
* 큰 샘플에서 조사했다 : 샘플사이즈가 크다.
* 정밀한 저울로 측정했다 : 데이터의 흩어짐(분산)이 작다.
* 중량의 평균값이 50g에서 크게 벗어난다 : 평균값의 차이가 크다.


In [1]:
# 수치 계산을 위한 라이브러리
import numpy as np
import pandas as pd
from scipy import stats

# 그래프를 그리기 위한 라이브러리
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 표시 자릿수 지정
%precision 3

# 그래프를 주피터 노트북에 그리기 위한 설정
%matplotlib inline

In [2]:
junk_food = pd.read_csv('/content/drive/MyDrive/python study/Statistics/dataset/3-8-1-junk-food-weight.csv')['weight']
junk_food.head()

0    58.529820
1    52.353039
2    74.446169
3    52.983263
4    55.876879
Name: weight, dtype: float64

 ### t검정 구현
 귀무가설 : 봉지과자의 평균중량은 50g이다.  
대립가설 : 봉지과자의 평균중량은 50g이 아니다.  

 p값이 0.05(유의수준 5%)보다 낮으면 귀무가설을 기각하여 봉지과자의 중량은 유의미하게 50g과 다르다고 주장할 수 있다.  

 t값 = (표본평균 - 비교대상값) / 표준오차

In [4]:
# 표본평균
mu = np.mean(junk_food)
mu

55.385

In [5]:
# 자유도
df = len(junk_food) - 1
df

19

In [6]:
# 표준오차 = 표준편차 / 샘플사이즈의 루트값
sigma = np.std(junk_food, ddof=1)
se = sigma / np.sqrt(len(junk_food))
se

1.958

In [7]:
# t값
t_value = (mu - 50) / se
t_value

2.750

#### P값 계산
모집단을 정규분포라고 가정했을 때 t값은 t분포를 따른다고 생각할 수 있으므로 t분포의 누적분포함수 사용  

모평균을 50이라고 가정했을 때의 t값이 표본의 t값보다 작을 확률을 alpha, 그 반대는 1-alpha

1-alpha가 작아지면 t값이 t(_표본)보다 클 확률이 낮다라는 말이 되어 유의미한 차이를 얻을 수 있게 됨  

양측검정이므로 p값은 (1-alpha) * 2

In [8]:
# p값
alpha = stats.t.cdf(t_value, df = df)
(1 - alpha) * 2

## p값이 유의수준 0.05보다 작으므로 유의미한 차이가 있다고 볼 수 있다.

0.013

In [9]:
# 함수 사용하여 t검정
stats.ttest_1samp(junk_food, 50)

Ttest_1sampResult(statistic=2.750339683171343, pvalue=0.012725590012524182)

#### 시뮬레이션에 의한 p값 계산
p값의 의미를 해석하기 위해 p값을 시뮬레이션으로 계산  

p값은 귀무가설이 옳다고 가정한 뒤 몇 번이고 표본추출을 하고 t값 계산을 반복했을 때, t(_표본)과 같거나 그보다 큰 t값을 얻는 비율

In [10]:
size = len(junk_food)
sigma = np.std(junk_food, ddof = 1)

In [11]:
t_value_array = np.zeros(50000)

In [13]:
np.random.seed(1)
norm_dist = stats.norm(loc = 50, scale = sigma)
for i in range(0, 50000):
  sample = norm_dist.rvs(size = size)
  sample_mean = np.mean(sample)
  sample_std = np.std(sample, ddof = 1)
  sample_se = sample_std / np.sqrt(size)
  t_value_array[i] = (sample_mean - 50) / sample_se

In [14]:
# 50000개의 t값 중 t(_표본)을 넘어선 비율
(sum(t_value_array > t_value) / 50000) * 2

0.013